# Adapted from SRC_params_JG.ipynb from Vaishali

---

This notebook is to optimise the parameters of the SRC for OzHF. 

Authors : V.Adya, B.Slagmolen, D.E.McClelland

Script adapted from work done by D,Töyrä and V.Adya for ET-LF FC optimisation

Objective : get a good set of starting numbers for OzHF sensitivity with detuned, long signal recycling cavity in a aLIGO like configuration.



# Import a bunch of packages 
Note: needs clean up

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pykat
import scipy

import scipy.io
import h5py


from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
#from pykat.parallel import parakat
from scipy.interpolate import interp1d
from scipy import optimize


matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12


%matplotlib inline

# Old Analytics

In [ ]:
# Reduced Planck's constant
hbar = 1.0545718e-34
# Speed of light
c = 299792458.0
# Boltzmann constant
kb = 1.38064852e-23
# Carrier wavelength
lam = 1.064e-6
# Test mass masses
m = 40#np.inf
# Carrier frequency
f0 = c/lam
# Carrier angular frequency
w0 = 2.0*np.pi*f0
# Number of frequency points
N = 500
# Frequency, max and min. 
F_min = 10
F_max = 1e5

# Frequency array
Fs = np.logspace(np.log10(F_min), np.log10(F_max), N)
# Angular frequency
Ws = 2.0*np.pi*Fs
# SQL
#SQL_h = np.sqrt((8.0*hbar)/(m*Ws**2*L**2))
####################################
# IFO PARAMETERS
####################################
# Beam splitter power
P_BS = 1300.0

# Lengths
# -------
L = 3994.469    # Arms
Lsrd = 56.01031  # SR

# Tunings
# --------
# Arm cavity mirrors
T_ITM = 0.014
R_ITM = 1 - T_ITM
t_itm = np.sqrt(T_ITM)
r_itm = np.sqrt(R_ITM)

# SRC mirrors
T_SRM = 0.2
R_SRM = 1 - T_SRM
t_srm = np.sqrt(T_SRM)
r_srm = np.sqrt(R_SRM)

# Other stuff
# -----------
# SRM tuning
phid = 0
# Free spectral range
FSR = c/(2.0*L)
# Arm bandwidth (angular frequency)
# gamma = (T_ITM*c)/(4.0*L)
gamma = 2.0*np.pi*(2.0*FSR/np.pi)*np.arcsin( (1.0-np.sqrt(1.0-T_ITM))/(2.0*(1.0-T_ITM)**(1.0/4.0) ) )/2.0

#gamma = 2*np.pi*66
#tau_arm  = L/c # Storage time in arm cavities
#tau_src = Lsr/c #Storage time in SRC
# Squeezing
# ------------------
Loss = 35e-6 #Internal losses in SRC
Loss2 = 5000e-6  # 0.5% loss in readout is 5000e-6 -- should correspond to about an order of magnitude
Loss3 = 50e-6 # squeezing Injection losses
rd = 0
phi_sqz = 0
rd2 = 0 #in dB
# ------------------

# AC matrix
# This matrix relates the DC carrier fields (a to k) to each other

M = np.zeros((27,27), dtype=complex)

np.fill_diagonal(M, 1)

# Indices for matrix
# DC Fields
# The propagation of the carrier field can is calculated independently from the sideband propagation
# because the carrier is not affected by the sidebands. This is done by populating a
# matrix, M, that relates the fields to each other. Fields a through k are assigned a number 0 through 18 
# that refer to their position in the matrix. The set of input-output relations for the carrier fields are

# compact assignment (added by James)
(a1, a2, b1, b2, c1, c2, d1, d2, e1, e2, f1, f2, g1, g2,
 l1, l2, k1, k2, lv1, lv2, v1, v2, lb1, lb2, n1, n2) = range(0, 26)
# GW signal
h = 26
def min_jet(lam=lam, ma=m, T_ITMs=T_ITM, P_BSs=P_BS, IS = Loss, EL = Loss2,
            IL = Loss3, T_SRMs = T_SRM, r2 = rd2, r = rd, L1=L, Lsr = Lsrd , phi = phid):
    
    if not isinstance(T_SRMs, np.ndarray):
        T_SRMs = np.array([T_SRMs])
    
    # Creating vectors to store quantum noise
    QN1 = np.zeros([len(Fs),len(T_SRMs)])
    QN2 = np.zeros([len(Fs),len(T_SRMs)])
    
    # Creating vectors to store interferometer response
    R_h1 = np.zeros([len(Fs),len(T_SRMs)])
    R_h2 = np.zeros([len(Fs),len(T_SRMs)])
   
    # Carrier frequency
    f0 = c/lam
    # Carrier angular frequency
    w0 = 2.0*np.pi*f0
    
    # ITM transmission
    T_ITMs = T_ITM
    R_ITM = 1 - T_ITM
    t_itm = np.sqrt(T_ITM)
    r_itm = np.sqrt(R_ITM)
    
    #gamma
    gamma = 2.0*np.pi*(2.0*FSR/np.pi)*np.arcsin((1.0-np.sqrt(1.0-T_ITMs))/(2.0*(1.0-T_ITMs)**(1.0/4.0)))/2.0
    
    for k,T_SRM in enumerate(T_SRMs):
        R_SRM = 1.0 - T_SRM;   # SRM Reflectivity
        t_srm = np.sqrt(T_SRM)
        r_srm = np.sqrt(R_SRM)

        for i,W in enumerate(Ws):

            # Locking/probe field relations

            # IFO input/output relation
            K = (8.0*w0*P_BSs)/(ma*L1**2*W**2*(gamma**2 + W**2 ))

            alpha = np.sqrt((2*w0*P_BSs)/((gamma**2 + W**2)*hbar))

            beta = np.arctan(W/gamma) 

            M[d1,c1] = np.exp(1j * 2.0 * beta) * 1
            M[d1,c2] = np.exp(1j  * 2.0 * beta) * 0
            M[d2,c1] = np.exp(1j  * 2.0 * beta) * -K
            M[d2,c2] = np.exp(1j  * 2.0 * beta) * 1

            M[d2,h] = np.exp(1j * beta) * alpha

            # SRM refl/trans relation
            M[b1,a1]  = np.sqrt(R_SRM) # b = rs*a + ts*f
            M[b1,f1]  = np.sqrt(T_SRM)
            M[b2,a2]  = np.sqrt(R_SRM)
            M[b2,f2]  = np.sqrt(T_SRM)

            M[e1,a1]  = np.sqrt(T_SRM) # e = ts*a - rs*d
            M[e1,f1]  = -np.sqrt(R_SRM)
            M[e2,a2]  = np.sqrt(T_SRM)
            M[e2,f2]  = -np.sqrt(R_SRM)

            # phase shift and rotation from free space 
            M[g1,k1]  = np.exp(1j*W*Lsr/c) * np.cos(phi)
            M[g1,k2]  = np.exp(1j*W*Lsr/c) * np.sin(phi)*-1
            M[g2,k1]  = np.exp(1j*W*Lsr/c) * np.sin(phi)
            M[g2,k2]  = np.exp(1j*W*Lsr/c) * np.cos(phi)

            M[f1,d1]  = np.exp(1j*W*Lsr/c)*np.cos(phi)
            M[f1,d2] = np.exp(1j*W*Lsr/c)*np.sin(phi)*-1
            M[f2,d1] = np.exp(1j*W*Lsr/c)*np.sin(phi)
            M[f2,d2] = np.exp(1j*W*Lsr/c)*np.cos(phi)

            # Coupling internal loss port
            M[k1,e1] = np.sqrt(1-IS);
            M[k1,l1] = np.sqrt(IS)
            M[k2,e2] = np.sqrt(1-IS)
            M[k2,l2] = np.sqrt(IS)

            # Squeezer relation
            M[c1,g1] = np.cosh(r) + np.sinh(r)*np.cos(2*phi_sqz)
            M[c1,g2] = np.sinh(r)*np.sin(2*phi_sqz)
            M[c2,g1] = np.sinh(r)*np.sin(2*phi_sqz)
            M[c2,g2] = np.cosh(r) - np.sinh(r)*np.cos(2*phi_sqz)   
            
            # Coupling external squeezing injection losses  
            M[a1,v1] = np.sqrt(1-IL);
            M[a1,lv1] = np.sqrt(IL)
            M[a2,v2] = np.sqrt(1-IL)
            M[a2,lv2] = np.sqrt(IL)

            # Coupling readout losses
            M[n1,b1] = np.sqrt(1-EL)
            M[n1,lb1] = np.sqrt(EL)
            M[n2,b2] = np.sqrt(1-EL)
            M[n2,lb2] = np.sqrt(EL)
            
            #inverse matrix

            #Inverting the matrix M gives the propagating fields in terms of the input fields. 
            #By inverting the matrix, we now have access to information about the amplitude 
            #and the phase of the fields at every location inside and outside the ifo.

            inM = np.linalg.inv(M)

            # Quantum noise
            QN1[i,k] = np.sqrt(((np.abs(inM[n1,v1])**2) * (10**(r2/10))) +
                               (np.abs(inM[n1,v2]))**2 * (10**(r2/10)) + 
                                np.abs(inM[n1,lb1])**2 + np.abs(inM[n1,lb2])**2 + 
                              np.abs(inM[n1,l1])**2 + np.abs(inM[n1,l2])**2 +
                               np.abs(inM[n1,lv1])**2 + np.abs(inM[n1,lv2])**2)

            QN2[i,k] = np.sqrt(((np.abs(inM[n2,v1])**2) * (10**(r2/10))) +
                               ((np.abs(inM[n2,v2])**2) * (10**(-r2/10))) + 
                                np.abs(inM[n2,lb1])**2 + np.abs(inM[n2,lb2])**2 + 
                              np.abs(inM[n2,l1])**2 + np.abs(inM[n2,l2])**2 +
                               np.abs(inM[n2,lv1])**2 + np.abs(inM[n2,lv2])**2)


            # GW transfer function         
            R_h1[i,k] = np.abs(inM[n1,h])
            R_h2[i,k] = np.abs(inM[n2,h])


            # Strain sensitivity
            #S1[z] = QN1[z]/R_h1[z];
            #S2[z] = QN2[z]/R_h2[z];


    S_1 = QN1/R_h1
    S_2 = QN2/R_h2
    
    return [QN1, QN2], [R_h1, R_h2], [S_1, S_2]


# min_jet(lam=lam, ma=m, T_ITMs=T_ITM, P_BSs=P_BS, IS = Loss, EL = Loss2,
#         IL = Loss3, T_SRMs = T_SRM, r2 = rd2, r = rd, L1=L, Lsr = Lsrd , phi = phid):
# QN_sn71, R_sn71, S_sn71 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.12,6,0.06,L,319,phid)

# no extenal squeezing
QN_sn71, R_sn71, S_sn71 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.12,0,0.06,L,319,phid)

# QN_sn11, R_sn11, S_sn11 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.325,6,0.0,L,Lsrd,phid)
# QN_sn21, R_sn21, S_sn21 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.325,0,0.0,L,Lsrd,phid)
# QN_sn31, R_sn31, S_sn31 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.325,10,0,L,2000,phid)
# QN_sn41, R_sn41, S_sn41 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.325,0.0,0.2,L,2000,phid)
# QN_sn51, R_sn51, S_sn51 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.325,10,0.2,L,2000,phid)
# QN_sn61, R_sn61, S_sn61 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.325,0,0,L,2000,phid)


# QN_sn81, R_sn81, S_sn81 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.12,6,0.0,L,319,phid)


# QN_sn91, R_sn91, S_sn91 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.12,0,0.06,L,319,phid)
# QN_sn711, R_sn711, S_sn711 = min_jet(lam,m,T_ITM,5.35e3,Loss,5000e-6,Loss3,0.12,6,0.0,L,319,0.01)


# QN_sn101, R_sn101, S_sn101 = min_jet(2e-6,200,0.001,3000,Loss,5000e-6,Loss3,0.05,7,0.0,L,56,phid)



# QN_t61, R_t61, S_t61 = min_jet(2e-6,211,0.01097,27000,0,0,0,0.048,10,0.0,2000,500,phid)
# QN_t611, R_t611, S_t611 = min_jet(2e-6,211,0.01097,27000,0,0,0,0.048,10,0.02,2000,500,phid)



Peak strain without squeezing :  2.42265695054583e-24
Peak strain with 10 dB injected squeezing :  8.0755231684861e-25
Input laser power : 500.0 W
PRC : 51.733415242578204 kW
Laser power incident on BS: 51.2651234909912 kW
X-arm cavity power : 5.1059676164704895 MW
Y-arm cavity power : 5.10596760689281 MW
SRC : 6.31847186925355e-05 W

# Tests with previously optimised values

In [ ]:
# mat1 = scipy.io.loadmat('/Users/vaishali/work/git/Optonumeric-modelling/Vaishali/OzHF/all_sens',
# squeeze_me=True)
# mat1 = scipy.io.loadmat('all_sens/AdV.mat',squeeze_me=True)
# mat2 = scipy.io.loadmat('all_sens/aLIGOdes.mat',squeeze_me=True)
# mat3 = scipy.io.loadmat('all_sens/aplus.mat',squeeze_me=True)
# # mat4 = scipy.io.loadmat('all_sens/ce.mat',squeeze_me=True)
# # mat5 = scipy.io.loadmat('all_sens/etd.mat',squeeze_me=True)
# # mat6 = scipy.io.loadmat('all_sens/kagra.mat',squeeze_me=True)
# # mat7 = scipy.io.loadmat('all_sens/voyager.mat',squeeze_me=True)
# oz_hf = h5py.File('all_sens/OzHF_2u_HP.hdf5','r')


# #list(oz_hf.keys())

# oz_hf_f  = oz_hf['Freq']
# oz_hf_f= np.array(oz_hf_f)
# oz_hf_qn = oz_hf['Total']
# oz_hf_qn = np.sqrt(np.abs(np.array(oz_hf_qn)))



# oz_hflP_f  = np.loadtxt('OzHF_NoiseBudget_lowP.txt')[:, 0]
# oz_hflP_qn = np.loadtxt('OzHF_NoiseBudget_lowP.txt')[:, 1]
# oz_hflP_qn = np.sqrt(np.abs(np.array(oz_hflP_qn)))

# aplus_f  = np.loadtxt('all_sens/aplus.txt')[:, 0]
# aplus_qn  = np.loadtxt('all_sens/aplus.txt')[:, 1]

# ce_f  = np.loadtxt('all_sens/ce.txt')[:, 0]
# ce_qn  = np.loadtxt('all_sens/ce.txt')[:, 1]

# et_f  = np.loadtxt('all_sens/et_d.txt')[:, 0]
# et_qn  = np.loadtxt('all_sens/et_d.txt')[:, 1]


# adv_f  = np.loadtxt('all_sens/advirgo.txt')[:, 0]
# adv_qn  = np.loadtxt('all_sens/advirgo.txt')[:, 1]

# voy_f  = np.loadtxt('all_sens/voyager.txt')[:, 0]
# voy_qn  = np.loadtxt('all_sens/voyager.txt')[:, 1]

# kag_f  = np.loadtxt('all_sens/kagra_sqz.txt')[:, 0]
# kag_qn  = np.loadtxt('all_sens/kagra_sqz.txt')[:, 1]


#scipy.io.savemat('ozhf_qnls_b.mat', {'ozhf_qnls_b': np.abs(y)})
#scipy.io.savemat('ozhf_f.mat', {'ozhf_f': out.x})

#ozhf_qnls_itm = scipy.io.savemat('ozhf_qnls_itm.mat', np.abs(y))
#ozhf_f = scipy.io.savemat('ozhf_f.mat', out_nbb.x)

fig1,ax = plt.subplots(dpi = 300)
# ax.plot(adv_f,adv_qn,lw=3,alpha=1.0,c='#ad03de',ls='-',label='AdV')
# #ax.plot(out4.x,(mat2['aLIGOdes']),'m',label='aLIGO design')
# #ax.plot(aplus_f,aplus_qn,lw=3,alpha=1.0,c='#15b01a',ls='-',label='A+')
# ax.plot(ce_f,ce_qn,lw=3,alpha=1.0,c='#fe0002',label='Cosmic explorer')
# ax.plot(et_f,et_qn,lw=3,alpha=1.0,c='#02ccfe',label='Einstein Telescope')
# #ax.plot(kag_f,kag_qn,'c',label='Kagra')
# ax.plot(voy_f,voy_qn,lw=3,alpha=1.0,c='k',label='Voyager')

# #ax.plot(out3.x,((np.abs(out3['NSR']))/2.5),
# #        lw=3,alpha=1.0,c='#929591',ls='--',label='1u Long NEMO broadband alt')
# ax.plot(oz_hf_f,oz_hf_qn,lw=3,alpha=1.0,c='darkseagreen',ls='-',label='NEMO')

#ax.plot(out5.x,((np.abs(out5['NSR']))),lw=3,alpha=1.0,c='khaki',ls='--',label='1u NEMO with long SRC')
ax.loglog(Fs,  S_sn71[1],'r' , lw=3,alpha=1.0,c='g',ls='--',label='1u NEMO with intSQZ')
#ax.plot(out4.x,((np.abs(out4['NSR']))/2.5),lw=3,alpha=1.0,c='#653700',ls='--', label='1u Long NEMO detuned')

#ax.plot(out9.x,((np.abs(out9['NSR']))),lw=3,alpha=1.0,c='#15b01a',ls='--',label='IS old')
#ax.plot(out6.x,((np.abs(out6['NSR']))/2.5),lw=3,alpha=1.0,c='blue',ls='--',label='Short SRC NEMO detuned')
#ax.plot(out7.x,((np.abs(out7['NSR'])))/2.5,lw=3,alpha=1.0,c='peachpuff',
#        ls='--',label='1u Long NEMO detuned w intSqz')


# ax.plot(out8.x,((np.abs(out8['NSR']))),lw=3,alpha=1.0,c='m',ls='--',label='LIS')

#ax.plot(oz_hflP_f,oz_hflP_qn,lw=3,alpha=1.0,c='peachpuff',ls='-',label='NEMO half power')

#ax.loglog(Fs, S_sn11[1],'m--' , linewidth=3, label='Advanced LIGO+ design')
#ax.loglog(Fs, S_sn31[1],'r', label='aLIGO long SRC + ES')
# #ax1.loglog(Fs, S_sn41[1],'m' , label='aLIGO + long SRC + IS')
# #ax1.loglog(Fs,  S_sn51[1],'b--' , label='aLIGO 2km SRC + IS + 10 dB ES')
# #ax1.loglog(Fs,  S_sn61[1],'b' , label='aLIGO long SRC with all losses')
#ax.loglog(Fs,  S_sn81[1],'k--' , linewidth=1,label='Advanced LIGO+ with long SRC')
# ax.loglog(Fs,  S_sn91[1],'r--' , linewidth=3,label='aLIGO-HF with no external squeezing')
# ax.loglog(Fs,  S_sn711[1],'g' , linewidth=3,label='test')


ax.set_yscale('log')
ax.set_xscale('log')
ax.set_axisbelow(True)
ax.grid(b=True,which='minor', color='k', linestyle='-')
ax.legend(loc='upper left',fontsize = 'x-small',ncol=2)
ax.set_xlim(100,5000)
ax.set_ylim(5E-25,7E-24)
ax.set_xlabel('Frequency [Hz]',fontsize=12)
ax.set_ylabel("Sensitivity [1/$\sqrt{Hz}$]",fontsize=12)
#plt.savefig('comparison_2km_bounded.eps')
plt.show(fig1)

print("peak sensitivity of {0:.3g} Hz^-0.5 obtained at {1:.5g} Hz".
      format(np.min(S_sn71[1]), Fs[np.argmin(S_sn71[1])]))